In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import networkx as nx
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
from node2vec import Node2Vec
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.linear_model import LinearRegression, HuberRegressor, Ridge, LogisticRegression, GammaRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from catboost import CatBoostRegressor, CatBoostClassifier 
from lazypredict.Supervised import LazyClassifier
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec
from gensim.models.word2vec import Word2Vec

from utils.threading import ThreadWithReturnValue

In [163]:
!pip freeze | grep gensim

gensim==4.2.0


In [3]:
data = pd.read_csv('datasets/Hepatotoxicity_Hepatotoxicity.csv')
data = data[['Canonical SMILES', 'Toxicity Value']].dropna()

### Trying out different methods to get embeddigns from our smiles

In [4]:
def sentences2vec(sentences, model, unseen='UNK', agg='mean'):
    keys = set(model.wv.index_to_key)
    vec = []
    for sentence in sentences:
        for word in sentence:
            if word not in keys:
                continue
            vec.append(model.wv[word])
    vec = np.array(vec)

    return vec

def get_mol_embedding(smiles):
    global w2v_model
    
    mol = Chem.MolFromSmiles(smiles)
    sentence = MolSentence(mol2alt_sentence(mol, radius=1))
    print(sentence)
    embedding = sentences2vec([sentence], w2v_model)
    
    return embedding

def get_graph_vector(graph: nx.Graph) -> np.ndarray:
    node2vec = Node2Vec(graph, dimensions=32, walk_length=15, num_walks=50, workers=6)
    model = node2vec.fit(window=10, min_count=1)

    node_embeddings = {str(node): model.wv[str(node)] for node in graph.nodes()}
    node_embeddings = np.array(list(node_embeddings.values()))

    graph_embedding = np.mean(node_embeddings, axis=0)

    return graph_embedding

def smiles_to_graph(smiles: str) -> nx.Graph:
    mol = Chem.MolFromSmiles(smiles)

    print('Reading smile: %s' % smiles)

    graph = nx.Graph()

    for atom in mol.GetAtoms():
        graph.add_node(atom.GetIdx(),
                       atomic_num=atom.GetAtomicNum(),
                       is_aromatic=atom.GetIsAromatic(),
                       atom_symbol=atom.GetSymbol())

    for bond in mol.GetBonds():
        graph.add_edge(bond.GetBeginAtomIdx(),
                       bond.GetEndAtomIdx(),
                       bond_type=bond.GetBondType())

    return graph

def get_morgan_fingerprint(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = np.array(RDKFingerprint(mol))

    return fingerprint

In [5]:
corpus = list(data['Canonical SMILES'].apply(lambda x: list(x)))

In [13]:
w2v_model = Word2Vec(corpus, vector_size=100, window=5, min_count=1, workers=4)
w2v_model.save("word2vec_for_smiles")

In [14]:
mol_fingerprints = data['Canonical SMILES'].apply(get_morgan_fingerprint).values

In [15]:
mol_graphs = data['Canonical SMILES'].apply(smiles_to_graph).values

Reading smile: C#CC1(OC(N)=O)CCCCC1
Reading smile: OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
Reading smile: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)Br
Reading smile: O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-]
Reading smile: FC(F)(F)C(F)(F)C(F)(F)F
Reading smile: FC(F)(F)C(Cl)Br
Reading smile: FC(F)OC(Cl)C(F)(F)F
Reading smile: FC(F)OC(F)C(F)(F)F
Reading smile: FC(F)OC(F)(F)C(F)Cl
Reading smile: O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO)C(=O)CO)c1I
Reading smile: O=[N+]([O-])O[C@H]1CO[C@H]2[C@@H]1OC[C@H]2O[N+](=O)[O-]
Reading smile: Nc1nc(=S)c2[nH]cnc2[nH]1
Reading smile: S=c1nc[nH]c2nc[nH]c12
Reading smile: Nc1[nH]c(=O)ncc1F
Reading smile: Clc1cc(Cl)c(OCC#CI)cc1Cl
Reading smile: O=c1[nH]cc(F)c(=O)[nH]1
Reading smile: O=c1[nH]cc(N(CCCl)CCCl)c(=O)[nH]1
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1I
Reading smile: NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1
Reading smile: Nc1nc(=O)c2c([nH]1)NCC(C

In [16]:
mol_w2v_embeddings = data['Canonical SMILES'].apply(get_mol_embedding, w2v_model).values

MolSentence with 24 words
MolSentence with 24 words
MolSentence with 52 words
MolSentence with 30 words
MolSentence with 22 words
MolSentence with 14 words
MolSentence with 20 words
MolSentence with 20 words
MolSentence with 20 words
MolSentence with 66 words
MolSentence with 32 words
MolSentence with 22 words
MolSentence with 20 words
MolSentence with 18 words
MolSentence with 28 words
MolSentence with 18 words
MolSentence with 30 words
MolSentence with 34 words
MolSentence with 34 words
MolSentence with 32 words
MolSentence with 69 words
MolSentence with 20 words
MolSentence with 40 words
MolSentence with 48 words
MolSentence with 34 words
MolSentence with 32 words
MolSentence with 34 words
MolSentence with 42 words
MolSentence with 46 words
MolSentence with 44 words
MolSentence with 64 words
MolSentence with 48 words
MolSentence with 34 words
MolSentence with 16 words
MolSentence with 18 words
MolSentence with 38 words
MolSentence with 34 words
MolSentence with 32 words
MolSentence 

In [17]:
mol_w2v_embeddings

array([array([], dtype=float64), array([], dtype=float64),
       array([], dtype=float64), ..., array([], dtype=float64),
       array([], dtype=float64), array([], dtype=float64)], dtype=object)

### Method to get embeddings from graph model

In [18]:
def get_graphs_embeddings(graphs):
    graph_embeddings = []
    
    for graph in tqdm(graphs):
        embedding = get_graph_vector(graph)
        graph_embeddings.append(embedding)
        
    return graph_embeddings

In [6]:
data = data.dropna()
data

,Canonical SMILES,Toxicity Value
0,C#CC1(OC(N)=O)CCCCC1,0
1,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,0
2,FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,1
4,O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-],1
5,FC(F)(F)C(F)(F)C(F)(F)F,0
...,...,...
2884,COc1cc(OC)c2oc3cc(OC)c(OC)c(OC4OC(OC5OCC(O)C(O...,0
2885,CC1(C)CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(OC...,0
2886,N=C(N)NCCN1CCCCCCC1,0
2887,OC1C(O)C(O)C(O)C(O)C1O,0


In [ ]:
thread_list, graph_embeddings = [], []
NUM_THREADS = 40

for batch in np.array_split(mol_graphs, NUM_THREADS):
    thread = ThreadWithReturnValue(target=get_graphs_embeddings, args=(batch,))
    thread_list.append(thread)
    thread.start()
    
for thread in thread_list:
    graph_embeddings.extend(thread.join())

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

...

Computing transition probabilities:   0%|          | 0/38 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/43 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/20 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/37 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(columns=['f%s' % i for i in range(32)] + [f'c%s' % i for i in range(len(mol_fingerprints[0]))])

for j, embedding in enumerate(graph_embeddings):
    row = {** {"f%s" % i: embedding[i] for i in range(len(embedding))},
           ** {"c%s" % i: mol_fingerprints[j][i] for i in range(len(mol_fingerprints[j]))}}
    df = df.append(row, ignore_index=True)

In [80]:
mol_fingerprints

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2       [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
5       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
2884    [1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, ...
2885    [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...
2886    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2887    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2888    [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...
Name: Canonical SMILES, Length: 2847, dtype: object

### Getting embeddings from graph visualisation of molecule from smiles

In [24]:
from karateclub import Graph2Vec

model = Graph2Vec()
graphs = data['Canonical SMILES'].apply(smiles_to_graph)
model.fit(graphs)
embeddings = model.get_embedding()
embd_df = pd.DataFrame(embeddings)

embd_df['toxicity_value'] = data['Toxicity Value']
embd_df = embd_df.dropna()
embd_df = embd_df.sample(frac=1).reset_index(drop=True)

Reading smile: C#CC1(OC(N)=O)CCCCC1
Reading smile: OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
Reading smile: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)Br
Reading smile: O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-]
Reading smile: FC(F)(F)C(F)(F)C(F)(F)F
Reading smile: FC(F)(F)C(Cl)Br
Reading smile: FC(F)OC(Cl)C(F)(F)F
Reading smile: FC(F)OC(F)C(F)(F)F
Reading smile: FC(F)OC(F)(F)C(F)Cl
Reading smile: O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO)C(=O)CO)c1I
Reading smile: O=[N+]([O-])O[C@H]1CO[C@H]2[C@@H]1OC[C@H]2O[N+](=O)[O-]
Reading smile: Nc1nc(=S)c2[nH]cnc2[nH]1
Reading smile: S=c1nc[nH]c2nc[nH]c12
Reading smile: Nc1[nH]c(=O)ncc1F
Reading smile: Clc1cc(Cl)c(OCC#CI)cc1Cl
Reading smile: O=c1[nH]cc(F)c(=O)[nH]1
Reading smile: O=c1[nH]cc(N(CCCl)CCCl)c(=O)[nH]1
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1I
Reading smile: NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1
Reading smile: Nc1nc(=O)c2c([nH]1)NCC(C

In [25]:
embd_df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,toxicity_value
0,0.03,0.01,-0.12,0.21,-0.01,-0.10,0.03,-0.13,0.01,-0.06,...,0.04,-0.00,-0.08,0.04,-0.06,-0.02,0.00,-0.00,0.02,0.00
1,0.02,0.00,-0.09,0.14,0.00,-0.08,0.03,-0.10,0.01,-0.04,...,0.03,-0.00,-0.06,0.03,-0.05,-0.01,0.01,0.00,0.00,0.00
2,0.03,0.01,-0.11,0.18,0.00,-0.10,0.03,-0.12,0.01,-0.05,...,0.04,-0.00,-0.08,0.04,-0.06,-0.03,0.00,-0.00,0.02,1.00
3,0.02,0.01,-0.10,0.17,-0.00,-0.10,0.04,-0.10,0.01,-0.05,...,0.03,-0.00,-0.08,0.04,-0.05,-0.03,0.01,0.00,0.02,0.00
4,0.02,0.01,-0.11,0.18,0.00,-0.10,0.03,-0.12,0.01,-0.05,...,0.03,-0.00,-0.07,0.03,-0.05,-0.02,0.01,-0.01,0.01,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2800,0.04,0.00,-0.12,0.19,-0.00,-0.10,0.03,-0.13,0.01,-0.05,...,0.03,-0.00,-0.09,0.04,-0.06,-0.03,0.01,-0.00,0.01,1.00
2801,0.02,0.01,-0.09,0.14,0.00,-0.07,0.03,-0.09,-0.00,-0.03,...,0.03,0.01,-0.05,0.03,-0.05,-0.01,0.01,0.00,0.00,0.00
2802,0.03,0.00,-0.11,0.16,-0.00,-0.09,0.04,-0.11,0.01,-0.05,...,0.03,-0.01,-0.07,0.04,-0.06,-0.02,0.01,-0.00,0.01,1.00
2803,0.03,0.00,-0.12,0.18,-0.01,-0.09,0.04,-0.12,-0.00,-0.05,...,0.03,-0.00,-0.07,0.03,-0.06,-0.02,0.00,-0.00,0.01,1.00


In [31]:
X, y = embd_df[range(0, 128)], embd_df.toxicity_value.values

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, shuffle=False)

### Using lazypredict to find best suitable model

In [34]:
from lazypredict.Supervised import LazyClassifier

lazy_clf = LazyClassifier(verbose=0, ignore_warnings=True)
models, preds = lazy_clf.fit(X_train, X_test, y_train, y_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:11<00:00,  2.62it/s]


In [35]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
CalibratedClassifierCV,0.57,0.56,0.56,0.57,2.18
LGBMClassifier,0.55,0.56,0.56,0.55,0.51
LinearDiscriminantAnalysis,0.56,0.56,0.56,0.56,0.33
RidgeClassifier,0.56,0.56,0.56,0.56,0.23
LogisticRegression,0.55,0.55,0.55,0.55,0.05
RidgeClassifierCV,0.55,0.55,0.55,0.55,0.43
SGDClassifier,0.54,0.55,0.55,0.54,0.09
LinearSVC,0.55,0.55,0.55,0.55,0.58
RandomForestClassifier,0.54,0.54,0.54,0.54,1.26


In [46]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB

base_clf = LogisticRegression()
calibrated_clf = CalibratedClassifierCV(base_clf, cv=5, n_jobs=-1)
calibrated_clf.fit(X_train, y_train)
preds = calibrated_clf.predict_proba(X)

preds.shape

(2805, 2)

In [52]:
data = data.dropna()

In [53]:
hepatotoxicity_preds = pd.DataFrame(data['Canonical SMILES'])

In [54]:
graphs = data['Canonical SMILES'].apply(smiles_to_graph)
model = Graph2Vec()
model.fit(graphs)

Reading smile: C#CC1(OC(N)=O)CCCCC1
Reading smile: OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO
Reading smile: FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)Br
Reading smile: O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-]
Reading smile: FC(F)(F)C(F)(F)C(F)(F)F
Reading smile: FC(F)(F)C(Cl)Br
Reading smile: FC(F)OC(Cl)C(F)(F)F
Reading smile: FC(F)OC(F)C(F)(F)F
Reading smile: FC(F)OC(F)(F)C(F)Cl
Reading smile: O=C(NCC(O)CO)c1c(I)c(C(=O)NCC(O)CO)c(I)c(N(CCO)C(=O)CO)c1I
Reading smile: O=[N+]([O-])O[C@H]1CO[C@H]2[C@@H]1OC[C@H]2O[N+](=O)[O-]
Reading smile: Nc1nc(=S)c2[nH]cnc2[nH]1
Reading smile: S=c1nc[nH]c2nc[nH]c12
Reading smile: Nc1[nH]c(=O)ncc1F
Reading smile: Clc1cc(Cl)c(OCC#CI)cc1Cl
Reading smile: O=c1[nH]cc(F)c(=O)[nH]1
Reading smile: O=c1[nH]cc(N(CCCl)CCCl)c(=O)[nH]1
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1F
Reading smile: O=c1[nH]c(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1I
Reading smile: NS(=O)(=O)c1cc(Cl)c(Cl)c(S(N)(=O)=O)c1
Reading smile: Nc1nc(=O)c2c([nH]1)NCC(C

In [55]:
embeddings = model.get_embedding()

In [56]:
embeddings.shape

(2847, 128)

In [58]:
probs = calibrated_clf.predict_proba(embeddings)
probs
hepatotoxicity_preds['Probability'] = [probs[i][1] for i in range(len(probs))]

In [62]:
hepatotoxicity_preds = hepatotoxicity_preds.reset_index(drop=True)

In [63]:
hepatotoxicity_preds.to_csv('hepatotoxicity-result.csv', index=True)

In [64]:
hepatotoxicity_preds

,Canonical SMILES,Probability
0,C#CC1(OC(N)=O)CCCCC1,0.50
1,OC[C@H](O)[C@@H](O)[C@H](O)[C@H](O)CO,0.49
2,FC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,0.52
3,O=[N+]([O-])OCC(CO[N+](=O)[O-])O[N+](=O)[O-],0.50
4,FC(F)(F)C(F)(F)C(F)(F)F,0.52
...,...,...
2842,COc1cc(OC)c2oc3cc(OC)c(OC)c(OC4OC(OC5OCC(O)C(O...,0.46
2843,CC1(C)CCC2(C(=O)O)CCC3(C)C(=CCC4C5(C)CC(O)C(OC...,0.48
2844,N=C(N)NCCN1CCCCCCC1,0.49
2845,OC1C(O)C(O)C(O)C(O)C1O,0.47
